In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import sys
sys.path.append('models') # append the directory containing the module

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import EncoderEEG
import deepcnn
import cnn2dlstm
import cnn1dlstm
import traintest
import warnings
warnings.filterwarnings('ignore')

In [49]:
import importlib
importlib.reload(traintest) # reload the module

<module 'traintest' from 'C:\\Users\\hp\\Desktop\\ML_Project Phase III\\ML_Project\\models\\traintest.py'>

In [25]:
torch.cuda.is_available()
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

GPU Name: NVIDIA GeForce MX450


In [40]:
dataset=torch.load(r"C:\Users\hp\Desktop\M.Tech\ML\Project Dataset\eeg_signals_raw_with_mean_std.pth")
labels=dataset['labels']
print(labels)

['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']


In [41]:
data=dataset['dataset']

In [42]:
splits=torch.load(r"C:\Users\hp\Desktop\M.Tech\ML\Project Dataset\block_splits_by_image_all.pth")

In [43]:
x_train,x_val,x_test,y_train,y_val,y_test=traintest.Splitter(data,splits,subject_no=0)

In [44]:
# For using 2D CNN or similar models, we need to add an extra dimension to our data for which we can use unsqueeze function

x_train=torch.stack(x_train)
#x_train=x_train.unsqueeze(3)
x_val=torch.stack(x_val)
#x_val=x_val.unsqueeze(3)
x_test=torch.stack(x_test)
#x_test=x_test.unsqueeze(3)
y_train=torch.tensor(y_train)
y_val=torch.tensor(y_val)
y_test=torch.tensor(y_test)

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val,y_val)
test_dataset = TensorDataset(x_test,y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16,shuffle=True)

In [47]:
num_classes = 40
learning_rate = 0.005
num_epochs = 50

model = EncoderEEG.EEGEncoder(num_classes=40).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

EEGEncoder(
  (temporal_block): TemporalBlock(
    (network): Sequential(
      (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(2,))
      (3): ReLU()
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(12,), dilation=(4,))
      (5): ReLU()
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(24,), dilation=(8,))
      (7): ReLU()
    )
  )
  (spatial_block): SpatialBlock(
    (network): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (res_blocks): ModuleList(
    (0-1): 2 x ResidualBlock(
      (conv1)

In [ ]:
#for inputs,labels in train_loader:
 #   print(inputs.shape)

In [50]:
# Call training function
traintest.train_model(model, train_loader, val_loader,criterion,optimizer,device,num_epochs)

Epoch 1/50, Training Loss: 3.6656, Validation Loss: 3.5767, Training Accuracy: 3.11%, Validation Accuracy: 3.35%
Epoch 2/50, Training Loss: 3.3977, Validation Loss: 3.3387, Training Accuracy: 5.18%, Validation Accuracy: 4.05%
Epoch 3/50, Training Loss: 2.9823, Validation Loss: 2.8068, Training Accuracy: 7.62%, Validation Accuracy: 9.11%
Epoch 4/50, Training Loss: 2.7596, Validation Loss: 2.7251, Training Accuracy: 10.45%, Validation Accuracy: 7.36%
Epoch 5/50, Training Loss: 2.6081, Validation Loss: 2.6347, Training Accuracy: 11.33%, Validation Accuracy: 13.26%
Epoch 6/50, Training Loss: 2.4959, Validation Loss: 2.4252, Training Accuracy: 13.27%, Validation Accuracy: 15.47%
Epoch 7/50, Training Loss: 2.2787, Validation Loss: 2.2356, Training Accuracy: 16.66%, Validation Accuracy: 17.57%
Epoch 8/50, Training Loss: 2.1831, Validation Loss: 2.1146, Training Accuracy: 19.42%, Validation Accuracy: 21.87%
Epoch 9/50, Training Loss: 2.0936, Validation Loss: 2.0679, Training Accuracy: 21.08%, 

In [51]:
traintest.evaluate_model(model,device,test_loader)

Test Accuracy: 85.88%


In [52]:
#Saving the model

torch.save(model.state_dict(), 'EEGEncoder_state_dict.pth')  # Save only the state dict
torch.save(model, 'EEGEncoder.pth')  # Save the entire model
torch.save(optimizer.state_dict(), 'optimizer_state_dict.pth')

## Not Relevant

In [ ]:
sample_input = torch.randn(16, 128, 440)  # Create a sample input tensor

# Initialize your model
num_classes = 40  # Adjust based on the number of classes in your task
model =EncoderEEG.EEGEncoder(num_classes)  # Use the correct model name here

# Pass the sample input through the model
try:
    output = model(sample_input)
    print("Output shape:", output.shape)  # Print the output shape
except Exception as e:
    print("An error occurred:", e)

An error occurred: Given groups=1, weight of size [64, 40, 3], expected input[16, 128, 440] to have 40 channels, but got 128 channels instead


In [ ]:
print(model.parameters)

<bound method Module.parameters of EEGEncoder(
  (temporal_block): TemporalBlock(
    (network): Sequential(
      (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(2,))
      (3): ReLU()
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(12,), dilation=(4,))
      (5): ReLU()
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(24,), dilation=(8,))
      (7): ReLU()
    )
  )
  (spatial_block): SpatialBlock(
    (network): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (res_blocks): ModuleList(
    (0-1